In [1]:
import sys
import cv2
import time
from queue import Queue
from threading import Thread


class VideoProcessingUtil:
    
    def __init__(self, videoFileLocation):
        
        # Initialize the stream with the location of the video file
        self.stream = cv2.VideoCapture(videoFileLocation)

        # Initialize stopped as False and transform to None 
        self.stopped = False
        self.transform = None

        # Initializing a queue of maximum size 10 to store the frames retrived from the video file
        self.Q = Queue(maxsize=10)

        # Initializing the thread with the target function
        self.thread = Thread(target=self.run, args=())
        
        # Initializing the daemon thread value as True
        self.thread.daemon = True
    
    
    def run(self):
        while True:
            
            # If thread has stopped, then break the loop
            if self.stopped:
                break

            # If the queue has NOT reached its maximum size or full capacity
            if not self.Q.full():
                
                # Read the next frame from the video file
                ret, frame = self.stream.read()

                # If all the frames in the video file have been read, then assign stopped as True
                if not ret:
                    self.stopped = True

                # Applying the transform on the thread that is retrieving the frames from the video file
                if self.transform:
                    frame = self.transform(frame)

                # Put the transformed frame in the frame queue
                self.Q.put(frame)
            
            # If the queue has reached its maximum size or full capacity
            else:
                
                # The thread should wait patiently for 20 milliseconds
                time.sleep(0.2)

        # Once all the frames are read, release the video file resources
        self.stream.release()

    
    def readQueue(self):
        
        # The next frame in the frame queue is returned for processing
        return self.Q.get()
    

    def next(self):
        
        # Check the frame queue to see if there are any more frames in it
        
        # A counter variable to track the number of attempts made
        numberOfAttempts = 0
        
        while self.Q.qsize() == 0 and not self.stopped and numberOfAttempts < 3:
            
            # Wait for 20 milliseconds if the queue is not empty, not stopped and the number of attempt is less than 3
            time.sleep(0.2)
            
            # Increase the value of the counter variable
            numberOfAttempts += 1

        # Return either True or False
        return self.Q.qsize() > 0

    
    def start(self):
        
        # Start a new thread
        self.thread.start()
        
        return self
    
    
    def stop(self):
        
        # Set stopped as True when this function is invoked
        self.stopped = True

        # Join the thread
        self.thread.join()
        
        
    def running(self):
        
        # Check to see if the thread that is retrieving the frames from the video file is active or not
        return self.next() or not self.stopped